<a href="https://colab.research.google.com/github/wyciszone/twitter_sentiment_analysis/blob/main/twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas scikit-learn nltk datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import re
import string
import pandas as pd
import nltk
from datasets import load_dataset
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

# data
dataset = load_dataset('carblacac/twitter-sentiment-analysis', trust_remote_code=True)

df_train = pd.DataFrame(dataset['train'])
df_test = pd.DataFrame(dataset['test'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# stop words
stop_words = set(stopwords.words('english'))
#lemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
# preprocessing
def preprocess_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # del urls
    text = re.sub(r'\@\w+|\#', '', text)  # del mentions and hashtags
    text = text.translate(str.maketrans('', '', string.punctuation))  # del punctuation
    tokens = word_tokenize(text)  # tokenize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # del stopwords & lemmatize
    return ' '.join(tokens)

In [ ]:
# apply
df_train['clean_text'] = df_train['text'].apply(preprocess_text)
df_test['clean_text'] = df_test['text'].apply(preprocess_text)

In [ ]:
# model
model = Pipeline([
    ('vectorizer', TfidfVectorizer()),  # Convert text into numerical TF-IDF features
    ('classifier', MultinomialNB())  # Naive Bayes classifier
])


In [ ]:
# train
model.fit(df_train['clean_text'], df_train['feeling'])

# predict
y_pred = model.predict(df_test['clean_text'])

In [ ]:
# evaluate
print(classification_report(df_test['feeling'], y_pred))
print(f"Accuracy: {accuracy_score(df_test['feeling'], y_pred)}")

              precision    recall  f1-score   support

           0       0.74      0.78      0.76     30969
           1       0.77      0.73      0.75     31029

    accuracy                           0.75     61998
   macro avg       0.75      0.75      0.75     61998
weighted avg       0.75      0.75      0.75     61998

Accuracy: 0.7515081131649408
